## SD v1.5 Resource Lists
**Homepage :** [SD Webui Complete setup](https://github.com/ffxvs/sd-webui-complete-setup)  
**Guide :** [Resource Lists Guide](https://github.com/ffxvs/sd-webui-complete-setup/wiki/Resource-Lists-Guide)

## 1. Requirements

### • Initialization
**REQUIRED EVERY TIME YOU START THE MACHINE**

In [ ]:
# Civitai Token - Required to download models that require login
civitai_token = ''


################################################################################################################

import os
os.chdir('/')
from internal.main import *

apply_envs1()
completed_message()

## 2. ControlNet

### • ControlNet Models

In [ ]:
# ControlNet - https://github.com/lllyasviel/ControlNet-v1-1-nightly
# Select models to download. Default models are : depth, ip-adapter, openpose, tile
controlnet = {
    'canny': boolean[0],
    'depth': boolean[1],
    'inpaint': boolean[0],
    'ip2p': boolean[0],
    'ipadapter': boolean[1],
    'ipadapter_face': boolean[0],
    'lineart': boolean[0],
    'lineart_anime': boolean[0],
    'mlsd': boolean[0],
    'normal': boolean[0],
    'openpose': boolean[1],
    'recolor': boolean[0],
    'scribble': boolean[0],
    'seg': boolean[0],
    'shuffle': boolean[0],
    'softedge': boolean[0],
    'tile': boolean[1]
}

# T2I-Adapter - https://github.com/TencentARC/T2I-Adapter/tree/SD
# A lighter alternative to offical controlnet models. Select models to download. 
t2i_adapter = {  
    'canny': boolean[0],
    'color': boolean[0],
    'depth': boolean[0],
    'keypose': boolean[0],
    'openpose': boolean[0],
    'seg': boolean[0],
    'sketch': boolean[0],
    'style': boolean[0],
    'zoedepth': boolean[0]
}


################################################################################################################

print("\n⏳ Download selected controlNet models...")
download_controlnet(controlnet, sd15_controlnet_url, sd)
print("\n⏳ Download selected T2I-Adapter...")
download_controlnet(t2i_adapter, sd15_t2i_adapter_url, sd)
os.chdir(webui)
completed_message()

### • Install from URLs

In [ ]:
# Install controlNet models from URL 
other_controlnet = []


########################################################################################

if other_controlnet:
    print('⏳ Downloading controlNet models...')
    other_resources(other_controlnet, f'{controlnet_models_path}/sd', civitai_token)

os.chdir(webui)
completed_message()

## 3. SD v1.5 Models
**Choose models you want to download**  
Some models hosted by Civitai require you to login to download.  
Create an API key in [Civitai user settings](https://civitai.com/user/account) then paste it to `civiai_token` inside Initialization cell.

### • Anime / Cartoon / 3D

In [ ]:
# Run this cell to display the model list

models_selection(sd15_anime_models_url, sd, civitai_token)
os.chdir(webui)

### • General Purpose

In [ ]:
# Run this cell to display the model list

models_selection(sd15_general_models_url, sd, civitai_token)
os.chdir(webui)

### • Realistic

In [ ]:
# Run this cell to display the model list

models_selection(sd15_realistic_models_url, sd, civitai_token)
os.chdir(webui)

### • Woman

In [ ]:
# Run this cell to display the model list

models_selection(sd15_woman_models_url, sd, civitai_token)
os.chdir(webui)

### • Install from URLs

In [ ]:
# Install models from URL 
other_models = []


########################################################################################

if other_models:
    print('⏳ Downloading models...')
    other_resources(other_models, f'{models_path}/sd', civitai_token)

os.chdir(webui)
completed_message()

## 4. LoRA, Embedding, Upscaler and VAE

### • Built-in Resources
**This is just for information. No need to run this cell.**

**Pre-installed LoRA :**
* [Add More Details - Detail Enhancer/Tweaker](https://civitai.com/models/82098)
* [Detail Tweaker](https://civitai.com/models/58390)
* [Epi Noise Offset](https://civitai.com/models/13941/epinoiseoffset)
* [LowRA](https://civitai.com/models/48139/lowra)

**Pre-installed Embeddings :**  
* [Embeddings Collection](https://huggingface.co/ffxvs/embeddings-collection)
* [Negative Prompts Pack](https://huggingface.co/ffxvs/negative-prompts-pack) 

**Pre-installed Upscaler :**  
* [UltraSharp](https://openmodeldb.info/models/4x-UltraSharp)

**Pre-installed VAE :**  
* [vae-ft-mse-840000](https://huggingface.co/stabilityai/sd-vae-ft-mse-original)

### • Resource Lists

In [ ]:
####################################### LORA ##########################################

# Depth of Field Slider - https://civitai.com/models/135380
dopSlider = boolean[0]

# Doll Likeness Series - https://huggingface.co/Kanbara/doll-likeness-series
dollLikeness = boolean[0]

# Hands SD v1.5 - https://civitai.com/models/200255?modelVersionId=228003
handsLora = boolean[0]

# Intricate Background & Background Blur - https://civitai.com/models/105732
bgDetails = boolean[0]

# LCM Lora SD v1.5 - https://huggingface.co/latent-consistency/lcm-lora-sdv1-5
lcmLora = boolean[0]


############################ EMBEDDINGS (TEXTUAL INVERSION) ############################

# SCG Emotions Pack - https://civitai.com/models/8860
scgEmotion = boolean[0]

# SCG Embedding Toolkit - https://civitai.com/models/8551
scgToolkit = boolean[0]

# Zovya Age Slider - https://civitai.com/models/65214
ageSlider = boolean[0]

# Zovya Gender Slider - https://civitai.com/models/89709
genderSlider = boolean[0]


###################################### UPSCALER ########################################

# BSRGAN - https://openmodeldb.info/models/4x-BSRGAN
bsrgan = boolean[0]

# LSDIRPlus - https://openmodeldb.info/models/4x-LSDIRplus
lsdirPlus = boolean[0]

# NMKD Superscale - https://openmodeldb.info/models/4x-NMKD-Superscale
superscale = boolean[0]

# Nomos8kSC - https://openmodeldb.info/models/4x-Nomos8kSC
nomos8ksc = boolean[0]

# Remacri - https://openmodeldb.info/models/4x-Remacri
remacri = boolean[0]

# UltraMix - https://upscale.wiki/w/index.php?title=Model_Database&oldid=1571
ultraMix = boolean[0]

# Valar - https://openmodeldb.info/models/4x-Valar
valar = boolean[0]


######################################## VAE ############################################

# Blessed2 - https://civitai.com/models/118561?modelVersionId=142467
blessed2 = boolean[0]

# ClearVAE - https://civitai.com/models/22354
clearVAE = boolean[0]

# kl-f8-anime2 - https://civitai.com/models/23906
kl_f8_anime2 = boolean[0]

# NAI/NovelAI - https://huggingface.co/WarriorMama777/OrangeMixs
novelAI = boolean[0]


###################################################################################################

os.chdir(webui)
download_builtin_resources(sd15_builtin_resources_url, sd)
download_lora(sd15_lora_list_url, sd)
download_embeddings(sd15_embedding_list_url, sd)
download_upscaler()
download_vae(sd15_vae_list_url, sd)
os.chdir(webui)
completed_message()

### • Install from URLs

In [ ]:
# LoRA
other_lora = []

# Embeddings
other_embeddings = []

# Upscaler
other_upscaler = []

# VAE
other_vae = []


###################################################################################################

download_other_resources(sd, other_lora, other_embeddings, other_upscaler, other_vae, civitai_token)
os.chdir(webui)
completed_message()